<a href="https://colab.research.google.com/github/ethvedbitdesjan/SummerResearch/blob/main/ResNetClassifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('content/')

Drive already mounted at content/; to attempt to forcibly remount, call drive.mount("content/", force_remount=True).


In [2]:
%cd /content/content/MyDrive/SummerResearch

/content/content/MyDrive/SummerResearch


In [3]:
# Critical imports
import os
import numpy as np
import pandas as pd
from PIL import Image
import random
import cv2
import copy

In [4]:
import torch
import torchvision
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.optim as optim
from torch.optim import lr_scheduler

In [5]:
funny_set = set()
not_funny_set = set()
with open('labelled_data/funny_combined1.csv', 'r') as f:
    lines = f.readlines()
    for line in lines:
        funny_set.add(line.strip())
with open('labelled_data/not_funny_combined1.csv', 'r') as f:
    lines = f.readlines()
    for line in lines:
        not_funny_set.add(line.strip())

In [6]:
train_data = []
train_funny_data = []
train_not_funny_data = []
count = 0
added = set()
for file in os.listdir('Funny_Originals_Final'):
  img_array = Image.open(os.path.join('Funny_Originals_Final', file))
  if file in funny_set:
    added.add(file)
    train_funny_data.append((img_array, 1))
  if file in not_funny_set:
    added.add(file)
    train_not_funny_data.append((img_array, 0))
  count += 1
for file in os.listdir('Non-Funny_Modified_Final'):
  img_array = Image.open(os.path.join('Non-Funny_Modified_Final', file))
  if file in funny_set:
    added.add(file)
    train_funny_data.append((img_array, 1))
  if file in not_funny_set:
    added.add(file)
    train_not_funny_data.append((img_array, 0))
random.seed(123456)

In [7]:
# print(len(train_funny_data), len(train_not_funny_data))
# train_not_funny_data1 = random.sample(train_not_funny_data, k = len(train_funny_data)+10)
# train_data = train_funny_data + train_not_funny_data1
# len(train_data), len(train_funny_data), len(train_not_funny_data), len(train_not_funny_data1)
train_data = train_funny_data + train_not_funny_data

In [8]:
random.shuffle(train_data)

In [9]:
class FunnyNotFunnyDataset(Dataset):
    def __init__(self, data, root_dir=None, transform=None):
        self.data = data
        self.root_dir = root_dir
        self.transform = transform
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        image = self.data[index][0]
        if self.transform:
          image = self.transform(image)
        label = self.data[index][1]
        label_tensor = torch.zeros(1)
        if label == 1:
          label_tensor[0] = 1
        return {'image_data':image, 'label':label_tensor}

In [10]:
idx_split = int(0.8 * len(train_data))
resnet50 = models.resnet50(pretrained=True)

# Freeze the weights of the ResNet model
for param in resnet50.parameters():
    param.requires_grad = False

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
num_classes = 1
new_classifier = torch.nn.Linear(resnet50.fc.in_features, num_classes)
resnet50.fc = new_classifier

In [12]:
IMAGE_SIZE = 224
NUM_CLASSES = 1103
BATCH_SIZE = 32
device = 'cuda' if torch.cuda.is_available() else 'cpu'
IMG_MEAN = [0.485, 0.456, 0.406]
IMG_STD = [0.229, 0.224, 0.225]
train_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(IMG_MEAN, IMG_STD)
])
test_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(IMG_MEAN, IMG_STD)
])
train_dataset = FunnyNotFunnyDataset(train_data[:idx_split], transform = train_transform)
test_dataset = FunnyNotFunnyDataset(train_data[idx_split:] ,transform = test_transform)

In [13]:
train_dataloader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle=True)
resnet50.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [14]:
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(resnet50.parameters(), lr=0.001)
lr_sch = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [15]:
import gc
gc.collect()

21

In [16]:
num_epochs = 10

for epoch in range(num_epochs):
  print('epoch:', epoch)
  lr_sch.step()
  resnet50.train()
  running_loss = 0.0
  total_loss = []
  lrs = []
  total_size = 0
  correct = 0
  for i, data in enumerate(train_dataloader):
    inputs = data['image_data'].to(device)
    label = data['label'].to(device)
    inputs = inputs.type(torch.cuda.FloatTensor)
    label = label.type(torch.cuda.FloatTensor)
    output = resnet50.forward(inputs)
    gc.collect()
    del inputs
    loss = criterion(output, label)
    loss.mean().backward()
    optimizer.step()
    optimizer.zero_grad()
    total_loss.append(torch.sum(loss))
    gc.collect()
    del loss
    torch.cuda.empty_cache()
    predictions = torch.as_tensor((output - 0.5) >= 0, dtype=torch.int32)
    correct += (predictions == label).float().sum().item()
    gc.collect()
    #gpu_usage()
    del predictions
    del label
    del output
    #gpu_usage()
    torch.cuda.empty_cache()
    #print(predictions, "\n", targets, "\n", correct)
    total_size += BATCH_SIZE
    accuracy = correct/(total_size)
    #print(correct, total_size)
    lrs.append(optimizer.param_groups[0]['lr'])
  print('Mean loss:', torch.mean(torch.stack(total_loss)), 'Accuracy:', accuracy, 'LRs:', lrs)

epoch: 0


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Mean loss: tensor(0.6658, device='cuda:0', grad_fn=<MeanBackward0>) Accuracy: 0.5831801470588235 LRs: [0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001]
epoch: 1
Mean loss: tensor(0.6210, device='cuda:0', grad_fn=<MeanBackward0>) Accuracy: 0.6102941176470589 LRs: [0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0

In [17]:
resnet50.eval()
total_size = 0
total_loss = []
correct = 0
with torch.no_grad():
    for data in test_dataloader:
        gc.collect()
        torch.cuda.empty_cache()
        inputs = data["image_data"].to(device)
        targets = data["label"].to(device)

        inputs = inputs.type(torch.cuda.FloatTensor)
        targets = targets.type(torch.cuda.FloatTensor)
        #print(ids.shape, "ids")
        batch_size = inputs.size(0)

        output = resnet50.forward(inputs)
        gc.collect()
        del inputs
        loss = criterion(output, targets)
        total_loss.append(torch.sum(loss))
        gc.collect()
        del loss
        torch.cuda.empty_cache()
        output = torch.sigmoid(output)

        predictions = torch.as_tensor((output - 0.5) > 0, dtype=torch.int32)
        if (predictions == targets).float().sum().item() > batch_size:
          print('error?')
        correct += (predictions == targets).float().sum().item()
        gc.collect()
        del predictions
        del targets
        del output
        torch.cuda.empty_cache()
        total_size += batch_size
        #gpu_usage()
    accuracy = correct/(total_size)
print("Validation Loss over a batch: {:.4f}; Training Loss over a single value: {:.4f} Accuracy: {:.2f}%".format(torch.mean(torch.stack(total_loss)),(torch.mean(torch.stack(total_loss)))/(batch_size*8), accuracy*100))
accuracy, correct, total_size

Validation Loss over a batch: 0.6303; Training Loss over a single value: 0.0029 Accuracy: 64.01%


(0.640074211502783, 345.0, 539)